In [15]:
import requests
from pymongo import MongoClient

In [16]:
# Client connects to "localhost" by default
client = MongoClient("mongodb://localhost:27017/")
# Create local "nobel" database on the fly
db = client["nobel"]

In [17]:
for collection_name in ["prizes"]:
    # collect the data from the API
    response = requests.get("http://api.nobelprize.org/v1/{}.json".format(collection_name[:-1]))
    # convert the data to json
    documents = response.json()[collection_name]
    # Create collections on the fly
    db[collection_name].insert_many(documents)

In [ ]:
prizes_collection = db["prizes"]

In [ ]:
# Count documents in a collection
n_prizes = db.prizes.count_documents(filter)
n_laureates = db.laureates.count_documents(filter)

In [ ]:
# Find one document to inspect
doc = db.prizes.find_one()

In [ ]:
# Count documents by providing afilter document to match
filter_doc = {
    'born': '1845-03-27',
    'diedCountry': 'Germany',
    'gender': 'male',
    'surname': 'Röntgen'
}
db.laureates.count_documents(filter_doc)

In [ ]:
## Simple filters
db.laureates.count_documents({'gender': 'female'})

In [ ]:
db.laureates.count_documents({'diedCountry': 'France'})

In [ ]:
db.laureates.count_documents({'bornCity': 'Warsaw'})

In [ ]:
filter_doc = {'gender': 'female',
'diedCountry': 'France',
'bornCity': 'Warsaw'}
db.laureates.count_documents(filter_doc)
## db.laureates.find_one(filter_doc)

### Query operators

In [ ]:
# Value in a range $in: <list>
db.laureates.count_documents({
    'diedCountry': {
    '$in': ['France', 'USA']
    }
})

In [ ]:
# Not equal $ne : <value>
db.laureates.count_documents({
    'diedCountry': {
    '$ne': 'France'
    }
})

## $ne return null values as well!

In [ ]:
# Comparison: > : $gt , ≥ : $gte | < : $lt , ≤ : $lte
db.laureates.count_documents({
    'diedCountry': {
    '$gt': 'Belgium',
    '$lte': 'USA'
    }
})

In [ ]:
db.laureates.find_one({
"firstname": "Walter",
"surname": "Kohn"})

In [ ]:
a = db.laureates.count_documents({
"prizes.affiliations.name": (
"University of California")})
b = db.laureates.count_documents({
"prizes.affiliations.city": (
"Berkeley, CA")})
print(f'Number of prizes in University of California is {a}. Number of prizes from Berkeley is {b}.')

In [ ]:
# $exists is to determine if a value exists in that field
db.laureates.count_documents({"bornCountry": {"$exists": False}})

In [ ]:
# at least 2 or more prizes won
db.laureates.count_documents({"prizes.1": {"$exists": True}})

In [ ]:
# same as unique in python
db.laureates.distinct("gender")

In [ ]:
db.laureates.distinct("prizes.category")

In [ ]:
# getting unique prize category with prize won with 4 people
db.laureates.distinct("prizes.category", {"prizes.share": '4'})

In [ ]:
# Prize categories with multi-winners
db.laureates.count_documents({"prizes.1": {"$exists": True}})

#lisitng that categories
db.laureates.distinct("prizes.category", {"prizes.1": {"$exists": True}})

In [ ]:
# $elemMatch
db.laureates.count_documents({"prizes": {"$elemMatch":{"category": "physics", "share": "1"}}})

In [ ]:
# Finding a substring with $regex
db.laureates.distinct("bornCountry",
{"bornCountry": {"$regex": "Poland"}})

# Beginning and ending (and escaping)
from bson.regex import Regex
db.laureates.distinct("bornCountry",{"bornCountry": Regex("^Poland")})

db.laureates.distinct(
"bornCountry",{"bornCountry": Regex("^Poland \(now")})

db.laureates.distinct(
"bornCountry",{"bornCountry": Regex("now Poland\)$")})

In [ ]:
## projection
# reducing data to fewer dimensions
# include only prizes.affiliations
# exclude _id
docs = db.laureates.find(
    filter={},
    projection={
        "prizes.affiliations": 1,"_id": 0 ## "_id" is included by default
        }
)

In [ ]:
# convert to list and slice
list(docs)[:3]

In [ ]:
# use "gender":"org" to select organizations
# organizations have no bornCountry
docs = db.laureates.find(
    filter={"gender": "org"},
    projection=["bornCountry", "firstname"])
list(docs)

# list the  elds to include ["field_name1", "field_name2"]

In [ ]:
# Sorting in-query with MongoDB
cursor = db.prizes.find({"category": "physics"}, ["year"], sort=[("year", 1)])
print([doc["year"] for doc in cursor][:5])

In [ ]:
for doc in db.prizes.find(
    {"year": {"$gt": "1966", "$lt": "1970"}},
    ["category", "year"],
    sort=[("year", 1), ("category", -1)]):
    print("{year} {category}".format(**doc))

In [ ]:
# creating index
db.prizes.create_index([("year", 1)])

In [ ]:
db.laureates.create_index([("firstname", 1), ("bornCountry", 1)])
db.laureates.find(
{"firstname": "Marie"}, {"bornCountry": 1, "_id": 0}).explain()

In [ ]:
for doc in db.prizes.find({"laureates.share": "3"}, limit=3):
    print("{year} {category}".format(**doc))

### Intro to Aggregation

In [ ]:
cursor = db.laureates.find(
    filter={"bornCountry": "USA"},
    projection={"prizes.year": 1},
    limit=3
)
for doc in cursor:
    print(doc["prizes"])

In [ ]:
cursor = db.laureates.aggregate([
    {"$match": {"bornCountry": "USA"}}, ## used to be filter
    {"$project": {"prizes.year": 1}}, ## used to be projection
    {"$limit": 3}
])
for doc in cursor:
    print(doc["prizes"])    

In [ ]:
## Adding sort and skip stages
from collections import OrderedDict
list(db.laureates.aggregate([
    {"$match": {"bornCountry": "USA"}},
    {"$project": {"prizes.year": 1, "_id": 0}},
    {"$sort": OrderedDict([("prizes.year", 1)])},
    {"$skip": 1},
    {"$limit": 3}
]))

In [ ]:
## count
list(db.laureates.aggregate([
    {"$match": {"bornCountry": "USA"}},
    {"$count": "n_USA-born-laureates"} ## creating your own key name
]))
# same as 
db.laureates.count_documents({"bornCountry": "USA"})

In [ ]:
### Field paths
# expression object {field1: <expression1>, ...}
db.laureates.aggregate([
    {"$project": {"n_prizes": {"$size": "$prizes"}}}
]).next()

In [ ]:
## How many prizes have been awarded in total?
list(db.laureates.aggregate([
    {"$project": {"n_prizes": {"$size": "$prizes"}}},
    {"$group": {"_id": None, "n_prizes_total": {"$sum": "$n_prizes"}}}
]))

In [ ]:
## sizing and summing
list(db.prizes.aggregate([
    {"$project": {"n_laureates": {"$size": "$laureates"},
    "year": 1, "category": 1, "_id": 0}}
]))

list(db.prizes.aggregate([
    {"$project": {"n_laureates": {"$size": "$laureates"},"category": 1}},
    {"$group": {"_id": "$category", "n_laureates":{"$sum": "$n_laureates"}}},
    {"$sort": {"n_laureates": -1}},
]))

### Unwind

In [ ]:
list(db.prizes.aggregate([
    {"$unwind": "$laureates"},
    {"$project": {"_id": 0, "year": 1, "category": 1,"laureates.surname": 1, "laureates.share": 1}},
    {"$limit": 3}
]))

In [ ]:
list(db.prizes.aggregate([
    {"$unwind": "$laureates"},
    {"$project": {"year": 1, "category": 1, "laureates.id": 1}},
    {"$group": {"_id": {"$concat": ["$category", ":", "$year"]},
    "laureate_ids": {"$addToSet": "$laureates.id"}}},
    {"$limit": 5}
]))

In [ ]:
list(db.prizes.aggregate([
    {"$project": {"n_laureates": {"$size": "$laureates"}, "category": 1}},
    {"$group": {"_id": "$category", "n_laureates": {"$sum": "$n_laureates"}}},
    {"$sort": {"n_laureates": -1}},
]))

In [ ]:
list(db.prizes.aggregate([
    {"$unwind": "$laureates"},
    {"$group": {"_id": "$category", "n_laureates": {"$sum": 1}}},
    {"$sort": {"n_laureates": -1}},
]))

### lookup

In [ ]:
list(db.prizes.aggregate([
    {"$match": {"category": "economics"}},
    {"$unwind": "$laureates"},
    {"$lookup": {"from": "laureates", "foreignField": "id","localField": "laureates.id", "as": "laureate_bios"}},
    {"$unwind": "$laureate_bios"},
    {"$group": {"_id": None,
            "bornCountries":{"$addToSet": "$laureate_bios.bornCountry"}
    }},
]))

## split and cond-itionally correct (with $concat)

In [ ]:
docs = list(db.laureates.aggregate([
    {"$match": {"died": {"$gt": "1700"}, "born": {"$gt": "1700"}}},
    {"$addFields": {"bornArray": {"$split": ["$born", "-"]},"diedArray": {"$split": ["$died", "-"]}}},
    {"$addFields": {"born": {"$cond": [{"$in": ["00", "$bornArray"]},
    {"$concat": [{"$arrayElemAt": ["$bornArray", 0]}, "-01-01"]},"$born"]}}},
    {"$project": {"died": {"$dateFromString": {"dateString": "$died"}},
                "born": {"$dateFromString": {"dateString": "$born"}},"_id": 0}}
]))